<a href="https://colab.research.google.com/github/dowrave/PythonToKaggle/blob/main/4_SanFranciscoCrimeClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### 모델을 돌리는 부분은 시간이 너무 오래걸려서 실행시키지 않고 커밋함.


In [ ]:
from google.colab import files
uploaded = files.upload()

!mkdir -p ~/.kaggle/
!mv kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c sf-crime
!unzip '*.zip'

In [ ]:
# 데이터 크기 확인
import os
DATA_PATH = "./"
for file in os.listdir(DATA_PATH):
  if 'csv' in file and 'zip' not in file:
    print(file.ljust(30) + str(round(os.path.getsize(file) / 1000000, 2)) + 'MB')

In [ ]:
import pandas as pd
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [ ]:
train.shape, test.shape

In [ ]:
train.info()

In [ ]:
test.info()

In [ ]:
# Train과 Test Feature의 차이 : Category, Descript, Resolution은 Train에만 있다
# Category는 종속변수 y
# Descript, Resolution은 개요, 마무리에 관한 글이나 테스트 데이터에 없기 때문에 제거해준다.

### 나머지 Feature에 대한 설명
- 시간 데이터 : Dates, DayOfWeek
- 공간 데이터 : PdDistrict, Addres, X, Y 
--> 시, 공간 데이터를 활용해 범죄의 유형을 예측하는 문제

### Submission의 형태
- 상기한 6개의 feature를 통해 범죄 유형을 39가지로 분류하라

In [ ]:
# sample_submission.csv
sample = pd.read_csv('sampleSubmission.csv')
sample.info()

In [ ]:
# 분류 결과값은 원핫 인코딩과 같이, 1개의 샘플 당 39개의 벡터가 있고 그 중 하나만 1이면 ㅇㅇ

### 다중분류 문제임
- Multi-Class Logarithm Loss..인데 결국 모든 샘플에 대한 다중 엔트로피 공식과 비슷함
- 차이라면 앞에 곱해지는 값이 p가 아니라 데이터가 속하는지 아닌지 여부인 y값(0 or 1 이라는 점?)
- https://www.kaggle.com/competitions/sf-crime/overview/evaluation
- 각 샘플에 대한 Loss 값의 합이 작을수록 좋은 모델


## 탐색적 자료 분석
- 데이터를 탐색할 때, 원래의 데이터를 복제해서 사용하는 것을 권장함
  - 원래 데이터에서 다시 출발해야 하는 경우가 생기기 때문

In [ ]:
train_df = train.copy()
train_df.shape == train.shape # 아니 책은 왜 자꾸 ==를 전부 =로 쓰냐고?

In [ ]:
def check_na(data):
  isnull_na = (data.isnull().sum() / len(data)) * 100
  data_na = isnull_na.drop(isnull_na[isnull_na == 0 ].index).sort_values(ascending = False)
  missing_data = pd.DataFrame({"Missing Ratio" : data_na,
                               "Data Type" : data.dtypes[data_na.index]})
  print("결측치 데이터 칼럼과 건수 : \n", missing_data)

check_na(train_df)

In [ ]:
# .duplicated : 중복된 데이터들을 띄워주는 듯. 
train_df.duplicated().sum()

In [ ]:
# 중복 데이터 제거
train_df.shape
train_df.drop_duplicates(inplace = True)
train_df.shape

In [ ]:
# 상기한 범죄 내용에 관한 텍스트 변수 2개 제거
train_df.drop(['Descript', 'Resolution'], axis = 1, inplace = True)
train_df.shape

In [ ]:
# 종속변수의 분포 확인
train_df['Category'].value_counts()[:5]

In [ ]:
train_df['Category'].value_counts()[-5:]

In [ ]:
temp = train_df.groupby('DayOfWeek').count().iloc[:, 0]
temp = temp.reindex([
                     'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'
])

print(temp)

In [ ]:
# 위 내용 시각화
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import cm # 이건 뭘까?
import numpy as np

fig, ax = plt.subplots(figsize = (10, 6))

ax = sns.barplot(
    temp.index, (temp.values / temp.values.sum())  * 100,
    orient = 'values'
)
ax.set_title('Incident Rates by DayOfWeek', fontdict = {'fontsize' : 16})
ax.set_xlabel('Weekday')
ax.set_ylabel('Incidents (%)')

plt.show()

- 그래프에서 볼 수 있는 것
--> 금요일이 제일 범죄 발생 많고 일요일은 적다


In [ ]:
# 요일별 범죄 유형 확인하기
""" 
사실 책을 따라가면서 기억해둬야 할 지점은 이런 곳들이다. 되게 당연한 듯이 하는데 직접 해보면 생각 안나는 곳들
특히 이거는 쓸 일이 많으니까 어떤 원리로 돌아가는지 이해해두는 게 좋을 듯!
"""
for idx, data in enumerate(train_df.groupby(['Category'])['DayOfWeek']):
  print(data) # 직관적으로 해석하자. groupby로 묶인 'Category' 및 'DayOfWeek'만 가진 항목임
              # 반복문은 각 'Category'에 대해 돌아감
              # data[0] 은 각 'Category'를, data[1]은 각 카테고리에 속하는 샘플들의 인덱스와 'DayOfWeek'을 같이 보여줌
  print("The Current Index : ", data[0])
  print(round(data[1].value_counts()  / data[1].count()*100 , 1) 
  )
  print()

  if idx == 1:
    break

In [ ]:
# 지역에 따른 범죄 수
temp = train_df.groupby('PdDistrict').count().iloc[:, 0]
print(temp)

In [ ]:
# 얘도 시각화
fig, ax = plt.subplots(figsize = (10, 6))

ax = sns.barplot(
    temp.index, (temp.values / temp.values.sum()) * 100,
    orient = 'v'
)
ax.set_title('Incident Rates by PdDistrict', fontdict = {'fontsize' : 16})
ax.set_xlabel('PdDistrict')
ax.set_ylabel('Incidents (%)')
plt.show()

In [ ]:
# 범죄 유형과 지역 조사
for idx, data in enumerate(train_df.groupby(['Category'])['PdDistrict']):
  print(data[0])
  print(round(data[1].value_counts() / data[1].count() * 100, 1))
  print()

  if idx == 1:
    break

- 지역에 따른 범죄 비율이 다름을 관찰할 수 있다
- 위도, 경도를 조사해보자

In [ ]:
fig, ax = plt.subplots(figsize = (10, 6))
sns.scatterplot(x= "X", y = "Y",
                data = train_df, alpha = 0.01, hue = "PdDistrict", ax = ax)
plt.show()

In [ ]:
"""
TENDERLOIN만 유독 동떨어져있는 걸 볼 수 있다.
그러나 바로 제거해서는 안됨 : 저게 많은 비중을 차지하는 데이터라면 성능에 영향을 줄 수 있기 때문이다.
따라서 저 데이터의 수가 얼마나 되는지부터 확인해본다
"""
print(train_df.loc[train.Y > 85].count()[0], train_df.loc[train.Y < 85].count()[0])

In [ ]:
# TENDERLOIN이 소수이기 때문에 제거해주고, 다시 산점도를 그려본다
train_df = train_df[train_df['Y'] < 90]

fig, ax = plt.subplots(figsize = (10, 6))
sns.scatterplot(x='X', y='Y',
                data = train_df, alpha = 0.01, hue = "PdDistrict", ax = ax)

plt.show()

In [ ]:
# 더 깊게 들어간다면, 범죄 유형을 세분화해서 살펴본다.
theft_df = train_df[train_df['Category'] == 'LARCENY/THEFT']
theft_df.shape

fig, ax = plt.subplots(figsize = (10 , 6))
sns.scatterplot(x = 'X', y = 'Y', data = theft_df, alpha = 0.01, hue = 'PdDistrict', ax = ax)
plt.show()

#### 날짜 데이터
- 반드시 먼저 날짜데이터의 타입을 확인하고 시작하자

In [ ]:
train_df['Dates'].dtypes 

In [ ]:
# Object니까 바꾸자
train_df.loc[: ,'Dates'] = pd.to_datetime(train_df['Dates'])
train_df.loc[: ,'Dates'].sample(1)

In [ ]:
# Datetime으로 데이터타입이 바뀌었다면 .dt.(뽑고 싶은 단위) 로 데이터를 뽑아낼 수 있음
train_df['Date'] = train_df.Dates.dt.date
train_df['Hour'] = train_df.Dates.dt.hour
daily_df = train_df.groupby('Date').count().iloc[:, 0]
daily_df

In [ ]:
train_df['Dates'].dtypes

In [ ]:
col = sns.color_palette()

fig, ax = plt.subplots(ncols = 2, figsize = (16, 6))
sns.lineplot(daily_df.index, daily_df.values, ax = ax[0])
ax[0].set_title('# of Incidents per day', fontdict = {"fontsize" : 16})
ax[0].set_ylabel("Incidents")

sns.kdeplot(data = daily_df, shade = True, ax = ax[1]) # ? 처음보는 거 같다
ax[1].axvline(x = daily_df.median(), ymax = 0.95, linestyle = '-', color = col[1])
ax[1].annotate(
    'Median: ' + str(daily_df.median()),
    xy = (daily_df.median(), 0.004),
    xytext = (200, 0.005), 
    arrowprops = dict(arrowstyle = '->', color = col[1], shrinkB = 10)
)
ax[1].set_title('Distribution of number of incidents per day', fontdict = {'fontsize' : 16})
ax[1].set_xlabel('Incidents')
ax[1].set_ylabel('Density')

plt.show()

In [ ]:
# 왼쪽 그래프 : 일자별 데이터의 이상치가 있음
# 이상치 조사
print("min Incidents of day : ", daily_df[daily_df.values == min(daily_df.values)])
print("max Incidents of day : ", daily_df[daily_df.values == max(daily_df.values)])

In [ ]:
# 왜 min값이 발생했는지, Max 값이 발생했는지 알 방법은 없다 그러나 이런 것들도 파악하는 게 데이터 분석가의 업무임

In [ ]:
# 하루 시간대별 범죄 발생율
hour_df = train_df.groupby('Hour').count().iloc[:, 0]
print('min incident hour : ', hour_df[hour_df.values == min(hour_df.values)])
print('max incident hour : ', hour_df[hour_df.values == max(hour_df.values)])

fig, ax = plt.subplots(figsize = (16, 5))
ax = sns.lineplot(x= hour_df.index, y = hour_df.values)
plt.title('Total Incidents per Hour')
plt.show()

- 위 그래프는 '전체' 데이터에 대한 것이기 때문에
- 범죄별로 들어가려면 별도로 데이터 뽑아서 확인하면 됨

## 피쳐 엔지니어링
- 하나의 함수로 만드는 것을 권장 
  -  공통으로 처리해야 하는 내용과, 데이터마다 전처리 성질이 조금 다른 것을 구분하는게 좋다

In [ ]:
def feature_engineering(data):

  # common
  data.drop_duplicates(inplace = True)
  data['Dates'] = pd.to_datetime(data['Dates'])
  # data['Date'] = pd.to_datetime(data['Dates'].dt.date) # 얘는 왜 기껏 뽑고 안쓰고 드랍함?
  data['DayOfWeek'] = data['Dates'].dt.weekday
  data['Month'] = data['Dates'].dt.month
  data['Year'] = data['Dates'].dt.year
  data['Hour'] = data['Dates'].dt.hour
  data.drop(columns = ['Dates', 'Address'], axis = 1, inplace = True)

  # 테스트 데이터의 Id 제거 및 훈련 데이터의 필요없는 column 제거
  if "Id" in data.columns:
    data.drop(['Id'], axis = 1 , inplace = True)
  else:
    data.drop(['Descript', 'Resolution'], axis = 1, inplace = True)

  return data


In [ ]:
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

In [ ]:
train = feature_engineering(train)
test = feature_engineering(test)

In [ ]:
train.shape, test.shape

### Tree 모델을 사용하므로 별도의 스케일링을 하지 앟는다
- 종속변수 PdDistrict에 Label Encoding을 진행한다

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

le1 = LabelEncoder()
train['PdDistrict'] = le1.fit_transform(train['PdDistrict']) # fit은 훈련 데이터에 맞춘다
test['PdDistrict'] = le1.transform(test['PdDistrict']) # 그래서 테스트는 transform만 한다

In [ ]:
le2 = LabelEncoder()
x = train.drop(columns = ['Category'])
y = le2.fit_transform(train['Category'])


## 모델링
- 모델링 구현 과정 : Baseline 모델 구축 -> 최적의 파라미터 찾기 -> 그럼에도 성능이 개선되지 않는다면 다시 피처 엔지니어링 / 기존 변수를 재활용함
- 이번 챕터는 GridSearchCV 를 통해 LightGBM, XGBoost, CatBoost 모형을 만듦

In [ ]:
# 기본 모델 구축
import lightgbm as lgb

train_set = lgb.Dataset(x, label = y, categorical_feature = ['PdDistrict'], free_raw_data = False)

params = {'objective' : "multiclass",
          'num_class' : 39}

lgbm_b0 = lgb.train(params, train_set, num_boost_round =6)
preds = lgbm_b0.predict(test)
preds

In [ ]:
# 기본 구축 모델로 제출
submission = pd.DataFrame(preds, columns = le2.classes_, index = test.index)
submission.to_csv('LGBM_base_model.csv', index_label = "Id")

In [ ]:
# !kaggle competitions submit -c sf-crime -f LGBM_base_model.csv -m 'BaseModel'

### 검증 데이터 만들고 GridSearchCV로 확인하기

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(x, y, test_size = 0.2, random_state = 42)
X_train.shape, X_val.shape, y_train.shape, y_val.shape

In [ ]:
# GridSearchCV : 대충 40분 이상 걸림
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMClassifier
import timeit
start_time = timeit.default_timer()
params = {'max_depth' : [5, 7],
          'min_child_weight' : [1, 3]}
lgbm = LGBMClassifier(objective = "multiclass", num_class = 39)
gridcv = GridSearchCV(estimator = lgbm,
                      param_grid = params,
                      n_jobs = -1,
                      verbose = 10,
                      cv = 2,
                      refit = True)

gridcv.fit(X_train, y_train, early_stopping_rounds = 5 , eval_set=[(X_train, y_train), (X_val, y_val)])

gridcv_df = pd.DataFrame(gridcv.cv_results_)
gridcv_df.loc[:, ['mean_test_score', 'params']]

terminate_time = timeit.default_timer()
print(terminate_time - start_time)

In [ ]:
# best parameter
gridcv.best_params_

In [ ]:
params = {'max_depth' : 7,
          'min_child_weight' : 1,
          'num_leaves' : 300,
          'objective' : 'multiclass',
          'num_class' : 39,
          'n_jobs' : -1}

lgtrain, lgval = lgb.Dataset(X_train, y_train), lgb.Dataset(X_val, y_val)
lgbmodel = lgb.train(params,
                     lgtrain,
                     num_boost_round = 100,
                     valid_sets = [lgtrain, lgval],
                     early_stopping_rounds = 10,
                     verbose_eval = True)

In [ ]:
preds = lgbmodel.predict(test)
submission = pd.DataFrame(preds, columns = le2.classes_, index = test.index)
submission.to_csv('LGBM_base_model.csv', index_label = "Id")

## 처음부터 GPU 켜고 lightGBM 쓰면 됐잖아?
- 관련 코드가 적혀 있는데 어떤 역할을 수행하는지 나와있지 않음

In [ ]:
# !git clone -recursive https://github.com/Microsoft/LightGBM
# %cd LightGBM
# !mkdir build
# !cmake-DUSE_GPU = 1
# !makje-j$(nproc)
# !sudo apt-get-y install python-pip
# !sudo-H pip install setuptools pandas numpy scipy scikit-learn-U
# %cd /content/LightGBM/python-package
# !sudo python setup.py install
# !pip3 uninstall scikit-learn
# !pip3 install scikit-learn == 0.21.3

In [ ]:
# 런타임 재시작 후 아래 코드 실행. 일단 위의 커맨드는 생략하고 GPU만 켜고 진행해봄
import LightGBM as lgb

train_set = lgb.Dataset(X, label = y, categorical_Feature = ['PdDistrict'], free_raw_data = False)

params = {'objective' : 'multiclass',
          'num_class' : 39,
          'device' : 'gpu'}

lgbm_b0 = lgb.train(params, train_set, num_boost_round = 0)

### XGBoost : RandomSearchCV까지 이용해봄

In [ ]:
import xgboost as xgb
import timeit
start_time = timeit.default_timer()

train_set = xgb.DMatrix(X, label = y)

params = {
    'objective' : 'multi:softprob'
    'num_class' : 39
}

xgb_b0 = xgb.train(params, train_set, num_boost_round = 6)

terminate_time = timeit.default_timer()
print(terminate_time)

test_xgb = xgb.DMatrix(test)
preds = xgb_b0.predict(test_xgb)
submission = pd.DataFrame(preds, columns = le2.classes_, index = test.index)
submission.to_csv('XGBoost_base_model.csv', index_label = 'Id')

In [ ]:
!kaggle competitions submit -c sf-crime -f XGBoost_base_model.csv -m 'XGBoost_base_model'

In [ ]:
# RandomizedSearchCV 활용
from sklearn.model_selection import RandomizedSearchCV

start_time = timeit.default_timer()

params = {'max_depth' : list(range(5, 7)),
          'min_child_weight' : list(range(1, 3))
          }

xgboost = xgb.XGBClassifier(objective = 'multi:softprob',
                            num_class = 39,
                            eval_metric = 'mlogloss')

randomcv = RandomizedSearchCV(estiamtor = xgboost,
                              param_distributions = params,
                              n_jobs = -1,
                              verbose = 10,
                              cv = 2,
                              random_state = 42,
                              refit = True)

randomcv.fit(X_train, y_train, early_stopping_rounds = 5, eval_set=[(X_train, y_train), (X_val, y_val)])
randomcv_df = pd.DataFrame(randomcv.cv_results_)
randomcv_df.loc[: , ['mean_test_score', 'params']]

terminate_time = timeit.default_timer()
print(terminate_time - start_time)

In [ ]:
# XGBoost는 LightGBM보다 학습 시간 소요가 많다 : 충분한 학습 시간을 가질 필요가 있음
print(randomcv.best_params_)

In [ ]:
# 찾은 최적의 하이퍼파라미터 값으로 모델 다시 설계
start_time = timeit.default_timer()

params = {'max_depth' : 6,
          'min_child_weight' : 1,
          'num_leaves' : 300,
          'objective' : 'multi:softprob',
          'num_class' : 39,
          'eval_metric' : 'mlogloss',
          'n_jobs' : -1}

xgbtrain, xgbval = xgb.DMatrix(X_train, y_train), xgb.DMatrix(X_val, y_val)
xgb_final_model = xgb.train(params, xgbtrain, 100, evals = [(xgbtrain, 'train'), (xgbval, 'eval')],
                            verbose_eval = 2)

terminate_time = timeit.default_timer()
print(terminate_time - start_time)

In [ ]:
# 결과 제출
test_xgb = xgb.DMatrix(test)
preds = xgb_final_model.predict(test_xgb)
submission = pd.DataFrame(preds,
                          columns = le2.classes_,
                          index = test.index)
submission.to_csv('XGBoost_final_model.csv', index_label = 'Id')

In [ ]:
!kaggle competitions submit -c sf-crime -f XGBoost_final_model.csv -m 'XGBoost_Final_Model'

In [ ]:
# 학습시간은 LightGBM보다 많지만 스코어는 조금 더 나음
# GPU를 쓰고 싶다면 파라미터에 Tree_Method, Gpu-Hist를 추가하면 됨
  # 모든 파라미터가 GPU를 활용하는 것이 아니기 때문에 공식 홈페이지에서 확인하고 사용할 것

### Catboost

In [ ]:
# 코랩에선 catboost를 지원하지 않는다고 한다 : 설치해줌
!pip install catboost

In [ ]:
# CatBoost는 입력 데이터로 변환 시 Pool을 사용한다(XGBoost에서는 DMatrix를 썼다)
from catboost import Pool, CatBoostClassifier

train_pool = Pool(data = X, label = y, cat_features = ['PdDistrict'])
cat_clf = CatBoostClassifier(iterations = 100, loss_function = 'MultiClass')

cat_clf.fit(train_pool)

In [ ]:
# 학습 후 예측값 뽑아서 제출
preds = cat_clf.predict_proba(test)
submission = pd.DataFrame(preds, columns = le2.classes_, index = test.index)
submission.to_csv('CatBoost_base_model.csv', index_label = 'Id')

In [ ]:
!kaggle competitions submit -c sf-crime -f XGBoost_final_model.csv -m "CatBoostBaseModel"

In [ ]:
# 최적의 하이퍼파라미터 뽑기
X_train, X_val, y_train, y_val = train_test_split(x, y, test_size = 0.2, random_state = 42)
X_train.shape, X_val.shape, y_train.shape, y_val.shape

In [ ]:
start_time = timeit.default_timer()

params= {'max_depth' : [5, 6],
         'learning_rate' : [0.03, 0.1]}

catboost = CatBoostClassifier(objective='MultiClass', iterations = 100)

grid_cv = GridSearchCV(estimator = catboost,
                       param_grid = params,
                       cv = 2,
                       refit = True)

grid_cv.fit(X_train, y_train, early_stopping_rounds = 5, eval_set=[(X_train, y_train), (X_val, y_val)])
grid_cv_df = pd.DataFrame(grid_cv.cv_results_)
grid_cv_df.loc[:, ['mean_test_score', 'params']]

terminate_time = timeit.default_timer()
print(terminate_time - start_time)

In [ ]:
print(grid_cv.best_params_)

In [ ]:
# 대충 최고의 파라미터 집어넣기
start_time = timeit.default_timer()
params = {'max_depth' : 6,
          'learning_rate' : 0.1,
          'min_data_in_leaf' : 3,
          'colsample_bylevel' : 0.8}

catboost_final = CatBoostClassifier(obejctive = 'MultiClass', iterations = 100, **params)

train_pool = Pool(data = X_train, label = y_train, cat_features = ['PdDistrict'])
test_pool = Pool(data = X_val, label = y_val, cat_features = ['PdDistrict'])

catboost_final.fit(train_pool, eval_set = test_pool)

terminate_time = timeit.default_timer()
print(terminate_time - start_time)

In [ ]:
results = catboost_final_evals_result_
results['learn']['Multiclass'][0:10]

In [ ]:
results['validation']['Multiclass'][0:10]

In [ ]:
# 그래프 작성
import matplotlib.pyplot as plt

epochs = len(results['validation']['Multiclass'])
x_axis = range(0, epochs)

fig, ax = plt.subplots(figsize = (10, 6))
ax.plot(x_axis, results['learn']['Multiclass'], label = 'Train')
ax.plot(x_axis, results['validation']['Multiclass'], label = 'Test')
ax.legend()
plt.ylabel("Log Loss")
plt.title('CatBoost Log Loss')
plt.show()

In [ ]:
# 과적합 발생 시 모형 재학습 필요
# 이 때 피쳐 엔지니어링부터 재시작하는 것이 좋음.
# 필자의 경우 변수의 수를 10개 이하로 줄이려고 함 - 여기서 많은 시간이 투자됨
# 간편한 방법은 L1, L2를 활용한 규제 및 Early_Stopping_rounds 파라미터를 활용함

In [ ]:
preds =  catboost_final.predict_proba(test)
submission = pd.DataFrame(preds,
                          columns = le2.classes_,
                          index = test_index)

submission.to_csv('CatBoost_final_model.csv', index_label = 'Id')

!kaggle competitions submit -c sf-crime -f CatBoost_final_model.csv -m "CatBoost_Final_Model"